In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password

In [2]:
# Study data files
imdb_movies_filepath = "data/IMDb movies.csv"
imdb_names_filepath = "data/IMDb names.csv"
imdb_ratings_filepath = "data/IMDb ratings.csv"

In [3]:
# Read the data into Pandas DataFrame
imdb_movies = pd.read_csv(imdb_movies_filepath)
imdb_names = pd.read_csv(imdb_names_filepath)
imdb_ratings = pd.read_csv(imdb_ratings_filepath)

# MOVIES FILE

In [4]:
imdb_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


In [7]:
movie_columns = ['imdb_title_id', 'title', 'year', 'duration', 'description', 'votes', 'avg_vote']
new_movie_df = imdb_movies[movie_columns].copy()
new_movie_df.head()

,imdb_title_id,title,year,duration,description,votes,avg_vote
0,tt0000574,The Story of the Kelly Gang,1906,70,True story of notorious Australian outlaw Ned ...,537,6.1
1,tt0001892,Den sorte drøm,1911,53,Two men of high rank are both wooing the beaut...,171,5.9
2,tt0002101,Cleopatra,1912,100,The fabled queen of Egypt's affair with Roman ...,420,5.2
3,tt0002130,L'Inferno,1911,68,Loosely adapted from Dante's Divine Comedy and...,2019,7.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,60,"An account of the life of Jesus Christ, based ...",438,5.7


In [10]:
# Rename the column headers
transform_movie_df = new_movie_df.rename(columns={'imdb_title_id': 'movie_id',
                                                 'title': 'movie_title',
                                                 'year': 'year_published',
                                                 'duration': 'movie_duration',
                                                 'votes': 'votes_count',
                                                 'avg_vote': 'votes_avg'})

# Clean the data by dropping duplicates and setting the index
transform_movie_df.drop_duplicates("movie_id", inplace=True)
transform_movie_df.set_index("movie_id", inplace=True)

transform_movie_df.head()

,movie_title,year_published,movie_duration,description,votes_count,votes_avg
movie_id,,,,,,
tt0000574,The Story of the Kelly Gang,1906,70,True story of notorious Australian outlaw Ned ...,537,6.1
tt0001892,Den sorte drøm,1911,53,Two men of high rank are both wooing the beaut...,171,5.9
tt0002101,Cleopatra,1912,100,The fabled queen of Egypt's affair with Roman ...,420,5.2
tt0002130,L'Inferno,1911,68,Loosely adapted from Dante's Divine Comedy and...,2019,7.0
tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,60,"An account of the life of Jesus Christ, based ...",438,5.7


In [11]:
transform_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81273 entries, tt0000574 to tt9914286
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movie_title     81273 non-null  object 
 1   year_published  81273 non-null  int64  
 2   movie_duration  81273 non-null  int64  
 3   description     78843 non-null  object 
 4   votes_count     81273 non-null  int64  
 5   votes_avg       81273 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 4.3+ MB


### Create database connection

In [12]:
# connection_string = f'{username}:{password}@localhost:5432/movies_db'
# engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
# engine.table_names()

### Load DataFrames into database

In [ ]:
# transform_movie_df.to_sql(name='movie', con=engine, if_exists='append', index=True)